# 程序框架

- 本教程目的在于教授 G4 的基本运行原理，以及如何快速应用到
- 刻意放弃一些类（功能）的使用，例如 SD，hist
- 模拟与数据处理分离
    - 为何模拟与数据处理必须分离？
    - G4程序只负责模拟部分（一次运行）
    - 基于 MakeClass 等框架的数据分析（可重复处理）

## 运行需要的基本类

- DetectorConstruction: 继承自G4VUserDetectorConstruction
- PhysicsList: 继承自G4VModularPhysicsList
- ActionInitialization: 继承自G4VUserActionInitialization
    - PrimaryGeneratorAction: 继承自G4VUserPrimaryGeneratorAction
    - RunAction:继承自G4UserRunAction
    - EventAction:继承自G4UserEventAction
    - TrackingAction:继承自G4UserTrackingAction
    - SteppingAction:继承自G4UserSteppingAction
    - StackingAction:继承自G4UserStackingAction

其中，**DetectorConstruction**，**PhysicsList**，**ActionInitialization**，**PrimaryGeneratorAction** 四个类不可缺少，其余几个类根据情况选择使用。

使用者必须熟练掌握探测器的几何设置 **DetectorConstruction** 和粒子源的设置 **PrimaryGeneratorAction** 两个类。

## 单位系统

G4可以进行单位换算，实质上是一些全局变量（G4SystemOfUnits.hh）


- 长度：um, mm, cm, m, km
- 能量：eV, keV, MeV
- 质量：mg, g, kg
- 摩尔质量: mole,
- 体积：mm3, cm3, m3, km3
- 时间：ns, s
- 电荷量：eplus

其中，默认单位为：mm, MeV, g, mole, ns, eplus，即如果没有写单位，则为默认单位。


例如:

```cpp
double x = 5.0*m;
double y = 4.0*m;
double z = 3.0*m;
```


- G4MTRunManager::Initialize()调用后，初始化G4内核
- G4MTRunManager::BeamOn()调用后，开始一次 G4Run

![111](img/RunEventTrackStep.png#pic_left)

- 一次BeamOn调用就是一次G4Run; 一次G4Run过程中，探测器设置和物理过程不可修改。
- BeamOn(1000) : 一次G4Run中，发射1000个事件（G4Event）
- 一个粒子的产生到该粒子停止追踪，径迹信息由同一个G4Track记录
- 在一个步长中，粒子信息该变量由G4Step记录



- G4Step包括两个“点”：PreStepPoint，PostStepPoint（对应class均为G4StepPoint）。
- PreStepPoint容纳物理过程发生前该粒子的状态（动能，电荷量等），
- PostStepPoint容纳物理过程发生后该粒子的状态。
- G4Step中保存的是这两“点”的粒子属性改变量（deltaE等）


- 步长：由该step的物理过程决定。
- 边界：step不能穿过两个物理体的边界，即边界上必须存在一个steppoint。此时step的status为fGeomBoundary

- G4Track：定义为粒子当前的状态（与PostStepPoint容纳的信息一致）。
- 每个G4Step调用后，G4Track信息更新。
- Track ID: ID编号从1开始，每个track中也记录其父粒子的track ID。
- 初级粒子（particlegun 发射的粒子）的父粒子Track ID为0



**提取物理量的接口：UserAction,  G4VSensitiveDetector**

- G4UserSteppingAction
    - UserSteppingAction(const G4Step*)
- G4UserTrackingAction
    - PreUserTrackingAction(const G4Track *)
    - PostUserTrackingAction(const G4Track *)
- G4UserEventAction
    - BeginOfEventAction()
    - EndOfEventAction()
- G4UserRunAction
    - BeginOfRunAction()
    - EndOfRunAction()


Example : 
- RunAction ：记录并输出程序运行时间、创建文件
- EventAction : 输出进度、SD得到该事件的所有数据
- TrackingAction : 输出每个track的初始信息、遗传标记、杀死次级粒子
- SteppingAction : 获得该step的所有数据


In [1]:
!jupyter nbconvert FRAME.ipynb --to html

[NbConvertApp] Converting notebook MAIN.ipynb to html

[NbConvertApp] Writing 573813 bytes to MAIN.html

